### Neste notebook, será criada a tabela "fact_wave_all" ao schema gold. Esta será a tabela fato do modelo.
### Em seguida, serão adicionadas ao schema gold as tabelas dimensão do modelo.

In [0]:
%python
spark.sql("USE CATALOG mvp_wvs")
spark.sql("USE SCHEMA gold")

### 1. Criação da tabela fato

In [0]:
-- Criação da tabela fato e disposição no schema gold
CREATE OR REPLACE TABLE mvp_wvs.gold.fact_wave_all AS
SELECT * FROM mvp_wvs.silver.wave_all;

In [0]:
-- Conferência da operação anterior (disposição da tabela fatono schema gold)
select * from mvp_wvs.gold.fact_wave_all

### 2. Criação das tabelas dimensão do modelo.

**2.1 dim_country**

In [0]:
%python
%pip install openpyxl

## Carregamento do arquivo excel com o código de países no schema staging
import pandas as pd

excel_path = '/Volumes/mvp_wvs/staging/wvs/CountrySpecificCodes.xlsx'
df_pd = pd.read_excel(
    excel_path,
    sheet_name='S003'
)

# Seleção da aba "S003" deste arquivo excel
df_pd = df_pd.astype(str)

df_spark = spark.createDataFrame(df_pd)

df_spark.write.format('delta').mode('overwrite').saveAsTable(
    'mvp_wvs.staging.country_specific_codes_s003'
)

display(df_spark)

In [0]:
-- Criação da tabela dim_country
CREATE or replace TABLE mvp_wvs.gold.dim_country USING DELTA AS
SELECT
  Code AS country_code, -- primary key
  Country
FROM
  mvp_wvs.staging.country_specific_codes_s003

In [0]:
-- Conferência do valor tipo "nan" da tabela dim_country
SELECT *
FROM mvp_wvs.gold.dim_country
WHERE Country IS NULL
   OR Country = 'nan';


In [0]:
--Exclusão do valor tipo "nan" da tabela dim_country
DELETE
FROM mvp_wvs.gold.dim_country
WHERE Country = 'nan';


In [0]:
-- Conferência da operação anterior
select *
from mvp_wvs.gold.dim_country

**2.2 Tabela "dim_age"**


In [0]:
-- Criação da tabela "dim_age"
CREATE OR REPLACE TABLE Dim_age AS
SELECT
  DISTINCT age,
  CASE
    WHEN age > 15 AND age < 26 THEN '15 - 25 years old'
    WHEN age > 25 THEN '> 25 years old'
    ELSE 'age not given'
  END AS age_range
FROM mvp_wvs.silver.wave_all;

-- a coluna "age" da tabela "dim_age" é primary key da coluna "age" da tabela "fact_wave_all"

**2.3 Tabela "dim_age_range"**

In [0]:
-- Criação da tabela "dim_age_range"
CREATE OR REPLACE TABLE dim_age_range AS
SELECT '15 - 25 years old' AS age_range
UNION ALL
SELECT '> 25 years old'
UNION ALL
SELECT 'age not given';

-- a coluna "age_range" da tabela "dim_age_range" é primary key da coluna "age_range" da tabela "dim_age"


**2.4 Tabela "dim_wave"**

In [0]:
-- Criação da tabela "dim_wave"
CREATE OR REPLACE TABLE dim_wave AS
SELECT 3 AS Wave
UNION ALL
SELECT 4
UNION ALL
SELECT 5
UNION ALL
SELECT 6
UNION ALL
SELECT 7;

-- A coluna "wave" da tabela "dim_wave" é primary key da coluna "wave_chronology" da tabela "fact_wave_all"